# MATH5470 Project: (Re-)Imag(in)ing Price Trends

This jupyter notebook is composed of 5 parts.

1. Data processing
2. Baseline model (including model construction, training and testing)
3. Sensitivity analysis
4. Grad-CAM
5. Regression model




## Data processing

In [ ]:
#Download and unzip the dataset.
!wget https://www.dropbox.com/s/njehqednn8mycze/img_data.zip?dl=0
!unzip img_data.zip?dl=0

Create and save training, validation, and testing datasets.

In [ ]:
import numpy as np
import  os 
import pandas as pd

#load the image and label dataset
file_list = os.listdir("/content/monthly_20d")
image_list = [i for i in file_list if 'dat' in i]
label_list = [i for i in file_list if 'feather' in i]
image_list.sort()
label_list.sort()

image_train_val, image_test = image_list[:7],image_list[7:] 
label_train_val, label_test = label_list[:7],label_list[7:] 


image_train_val_arr, image_test_arr = [], []
label_train_val_arr, label_test_arr = [], [] 
for i, j in zip(image_train_val, label_train_val):    
    image_train_val_arr.append(np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60)))
    label_train_val_arr.append(pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values)

for i, j in zip(image_test, label_test):
    image_test_arr.append(np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60)))
    label_test_arr.append(pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values)

Code for cold start if the data haven't been processed.

In [ ]:
cold_start = False

if cold_start:
  image_train_val_data = np.concatenate(image_train_val_arr, 0)
  image_train_val_data[image_train_val_data==255]=1
  label_train_val_data = np.concatenate(label_train_val_arr, 0)

  image_train_data, image_val_data = image_train_val_data[:int(0.7*len(image_train_val_data))], image_train_val_data[int(0.7*len(image_train_val_data)):]
  label_train_data, label_val_data = label_train_val_data[:int(0.7*len(label_train_val_data))], label_train_val_data[int(0.7*len(label_train_val_data)):]

  np.save('/content/drive/MyDrive/MATH5470/dataset/train_x.npy', image_train_data)
  np.save('/content/drive/MyDrive/MATH5470/dataset/train_y.npy', label_train_data)
  np.save('/content/drive/MyDrive/MATH5470/dataset/val_x.npy', image_val_data)
  np.save('/content/drive/MyDrive/MATH5470/dataset/val_y.npy', label_val_data)

  image_test_data = np.concatenate(image_test_arr, 0)
  image_test_data[image_test_data==255]=1
  label_test_data = np.concatenate(label_test_arr, 0)

  np.save('/content/drive/MyDrive/MATH5470/dataset/test_x.npy', image_test_data)
  np.save('/content/drive/MyDrive/MATH5470/dataset/test_y.npy', label_test_data)

else:
  image_train_data = np.load("/content/drive/MyDrive/MATH5470/dataset/train_x.npy")
  image_val_data = np.load("/content/drive/MyDrive/MATH5470/dataset/val_x.npy")
  label_train_data = np.load("/content/drive/MyDrive/MATH5470/dataset/train_y.npy")
  label_val_data = np.load("/content/drive/MyDrive/MATH5470/dataset/val_y.npy")
  image_test_data = np.load("/content/drive/MyDrive/MATH5470/dataset/test_x.npy")
  label_test_data = np.load("/content/drive/MyDrive/MATH5470/dataset/test_y.npy")

In [ ]:
print("The size of training image is " + str(image_train_data.shape))
print("The size of training label is " + str(label_train_data.shape))
print("The size of validation image is " + str(image_val_data.shape))
print("The size of validation label is " + str(label_val_data.shape))
print("The size of testing image is " + str(image_test_data.shape))
print("The size of testing label is " + str(label_test_data.shape))

DataLoader

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch

class ImageDataset(Dataset):
    def __init__(self, data_file_path, label_file_path, binary=True):
        self.data = np.load(data_file_path)
        self.label = np.load(label_file_path)
        self.binary = binary
    def __getitem__(self, index):
        x = self.data[index]
        y = self.label[index]
        x = torch.from_numpy(x).float()
        x = x.unsqueeze(0)
        if self.binary:
            y = np.where(y > 0, 1, 0)
        y = torch.from_numpy(y).float()
        return x, y
    
    def __len__(self):
        return len(self.data)

class ImgDataset(Dataset):
    def __init__(self, data_file, label_file, binary=True):
        self.data = data_file
        self.label = label_file
        self.binary = binary
    def __getitem__(self, index):
        x = self.data[index]
        y = self.label[index]
        x = torch.from_numpy(x).float()
        x = x.unsqueeze(0)
        if self.binary:
            y = np.where(y > 0, 1, 0)
        y = torch.from_numpy(y).float()
        return x,y
    
    def __len__(self):
        return len(self.data)

## CNN baseline model

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
class ConvNet(nn.Module):
    """Encoder for feature embedding"""
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 64, kernel_size=(5,3), padding=(5, 3), stride=(3,1) ,dilation=(2,1)),
                        nn.BatchNorm2d(64),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d(2,1))
        nn.init.xavier_uniform_(self.layer1[0].weight)
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,128,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(128),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,1)),)
        nn.init.xavier_uniform_(self.layer2[0].weight)
        self.layer3 = nn.Sequential(
                        nn.Conv2d(128,256,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(256),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,1)),)
        nn.init.xavier_uniform_(self.layer3[0].weight)
        self.fc = nn.Linear(18176, 2)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        result = self.fc(out)
        return result

def conv3():
    return ConvNet()

### Pretraining of the model

In [ ]:
import torch
import numpy as np
import random
import os
from copy import deepcopy
import math
import torch.nn as nn
import torch.nn.functional as F


class Averager():

    def __init__(self):
        self.n = 0
        self.v = 0

    def add(self, x):
        self.v = (self.v * self.n + x) / (self.n + 1)
        self.n += 1

    def item(self):
        return self.v


def pretrain(model, loader, optimizer, device):
    model.train()
    ## training with ce
    loss_avg = Averager() 
    for batch_idx, batch in enumerate(loader):
        model.zero_grad()
        optimizer.zero_grad()
        train_inputs, train_targets = batch[0], batch[1]
        train_targets = train_targets.long()
        train_inputs = train_inputs.to(device=device)
        train_targets = train_targets.to(device=device)
        train_logits = model(train_inputs)
        loss = nn.CrossEntropyLoss()(train_logits, train_targets)
        loss_avg.add(loss.item())
        loss.backward()
        optimizer.step()
    print("Train Loss %.4f" % (loss_avg.item()))
    return loss_avg.item()

def evaluate_batch(model, data_loader, device):
    model.eval()
    correct = num = 0
    for iter, pack in enumerate(data_loader):
        data, target = pack[0].to(device), pack[1].to(device)
        targets = target.long()
        logits = model(data)
        _, pred = logits.max(1)
        correct += pred.eq(target).sum().item()
        num += data.shape[0]
    torch.cuda.empty_cache()
    model.train()
    return correct / num

### Training

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import  os 

trainset = ImageDataset("/content/drive/MyDrive/MATH5470/dataset/train_x.npy","/content/drive/MyDrive/MATH5470/dataset/train_y.npy")
valset = ImageDataset("/content/drive/MyDrive/MATH5470/dataset/val_x.npy","/content/drive/MyDrive/MATH5470/dataset/val_y.npy")
batch_size = 64
epoch = 15
lr = 1e-3
wd =  5e-4
model_name = 'weight.pth'
train_loader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True, num_workers=4,
                pin_memory=True, drop_last=True)
val_loader = DataLoader(dataset=valset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)

device = 'cuda'
model = conv3().cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break


In [ ]:
import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

## Alternate CNN models and sensitivity analysis

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
class AltConvNet(nn.Module):
    """Encoder for feature embedding"""
    def __init__(self, dropout_rate=0.5, Xavier=True, pool_size=(2,1),filter_size=(5,3),dilation=(2,1), stride=(3,1)):
        super(AltConvNet, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 64, kernel_size=(5,3), padding=(5,3), stride=stride ,dilation=dilation),
                        nn.BatchNorm2d(64),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((pool_size)))
        if Xavier:
          nn.init.xavier_uniform_(self.layer1[0].weight)
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,128,kernel_size=(5,3),padding=(5,3), stride=stride ,dilation=dilation),
                        nn.BatchNorm2d(128),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((pool_size)),)
        if Xavier:
          nn.init.xavier_uniform_(self.layer2[0].weight)
        self.layer3 = nn.Sequential(
                        nn.Conv2d(128,256,kernel_size=(5,3),padding=(5,3), stride=stride ,dilation=dilation),
                        nn.BatchNorm2d(256),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((pool_size)),)
        if Xavier:
          nn.init.xavier_uniform_(self.layer3[0].weight)
        self.fc = nn.Linear(18432, 2)
        self.dropout = nn.Dropout(p=dropout_rate)




    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        result = self.fc(out)
        return result

def Altconv3(dropout_rate=0.5, Xavier=True, pool_size=(2,1),filter_size=(5,3),dilation=(2,1), stride=(3,1)):
    return AltConvNet(dropout_rate, Xavier, pool_size,filter_size,dilation, stride)

In [ ]:
model = Altconv3(dropout_rate=0).cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break

import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

In [ ]:
model = Altconv3(dropout_rate=0.25).cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break

import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

In [ ]:
device = 'cuda'
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
model = Altconv3(dropout_rate=0.75).cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break

import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

In [ ]:
model = Altconv3(Xavier=False).cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break

import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import  os 

trainset = ImageDataset("/content/drive/MyDrive/MATH5470/dataset/train_x.npy","/content/drive/MyDrive/MATH5470/dataset/train_y.npy")
valset = ImageDataset("/content/drive/MyDrive/MATH5470/dataset/val_x.npy","/content/drive/MyDrive/MATH5470/dataset/val_y.npy")
batch_size = 64
epoch = 15
lr = 1e-3
wd =  5e-4
model_name = 'weight.pth'
train_loader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True, num_workers=4,
                pin_memory=True, drop_last=True)
val_loader = DataLoader(dataset=valset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
device = 'cuda'
model = Altconv3(filter_size=(3,3)).cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break

In [ ]:
import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

In [ ]:
model = Altconv3(filter_size=(7,3)).cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break

import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

### Try different number of filter in the first layer.

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
class AltFilterConvNet(nn.Module):
    """Encoder for feature embedding"""
    def __init__(self):
        super(AltFilterConvNet, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 32, kernel_size=(5,3), padding=(5, 3), stride=(3,1) ,dilation=(2,1)),
                        nn.BatchNorm2d(32),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d(2,1))
        nn.init.xavier_uniform_(self.layer1[0].weight)
        self.layer2 = nn.Sequential(
                        nn.Conv2d(32,128,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(128),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,1)),)
        nn.init.xavier_uniform_(self.layer2[0].weight)
        self.layer3 = nn.Sequential(
                        nn.Conv2d(128,256,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(256),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,1)),)
        nn.init.xavier_uniform_(self.layer3[0].weight)
        self.fc = nn.Linear(18176, 2)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        result = self.fc(out)
        return result

def AltFilterconv3():
    return AltFilterConvNet()

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import  os 

trainset = ImageDataset("/content/drive/MyDrive/MATH5470/dataset/train_x.npy","/content/drive/MyDrive/MATH5470/dataset/train_y.npy")
valset = ImageDataset("/content/drive/MyDrive/MATH5470/dataset/val_x.npy","/content/drive/MyDrive/MATH5470/dataset/val_y.npy")
batch_size = 32
epoch = 15
lr = 1e-3
wd =  5e-4
model_name = 'weight.pth'
train_loader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True, num_workers=4,
                pin_memory=True, drop_last=True)
val_loader = DataLoader(dataset=valset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
device = 'cuda'
model = AltFilterconv3().cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break

import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

In [ ]:
class AltFilterConvNet(nn.Module):
    """Encoder for feature embedding"""
    def __init__(self):
        super(AltFilterConvNet, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 128, kernel_size=(5,3), padding=(5, 3), stride=(3,1) ,dilation=(2,1)),
                        nn.BatchNorm2d(128),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d(2,1))
        nn.init.xavier_uniform_(self.layer1[0].weight)
        self.layer2 = nn.Sequential(
                        nn.Conv2d(128,128,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(128),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,1)),)
        nn.init.xavier_uniform_(self.layer2[0].weight)
        self.layer3 = nn.Sequential(
                        nn.Conv2d(128,256,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(256),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,1)),)
        nn.init.xavier_uniform_(self.layer3[0].weight)
        self.fc = nn.Linear(18176, 2)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        result = self.fc(out)
        return result

def AltFilterconv3():
    return AltFilterConvNet()

In [ ]:
batch_size = 128
epoch = 15
lr = 1e-3
wd =  5e-4
model_name = 'weight.pth'
train_loader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True, num_workers=4,
                pin_memory=True, drop_last=True)
val_loader = DataLoader(dataset=valset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
device = 'cuda'
model = AltFilterconv3().cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break

import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

### Try different layers.

In [ ]:
class AltFilterConvNet(nn.Module):
    """Encoder for feature embedding"""
    def __init__(self):
        super(AltFilterConvNet, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 64, kernel_size=(5,3), padding=(5, 3), stride=(3,1) ,dilation=(2,1)),
                        nn.BatchNorm2d(64),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d(2,1))
        nn.init.xavier_uniform_(self.layer1[0].weight)
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,128,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(128),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,1)),)
        nn.init.xavier_uniform_(self.layer2[0].weight)
        self.layer3 = nn.Sequential(
                        nn.Conv2d(128,256,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(256),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,1)),)
        nn.init.xavier_uniform_(self.layer3[0].weight)
        self.fc = nn.Linear(34304, 2)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        result = self.fc(out)
        return result

def AltFilterconv3():
    return AltFilterConvNet()

In [ ]:
batch_size = 64
epoch = 15
lr = 1e-3
wd =  5e-4
model_name = 'weight.pth'
train_loader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True, num_workers=4,
                pin_memory=True, drop_last=True)
val_loader = DataLoader(dataset=valset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
device = 'cuda'
model = AltFilterconv3().cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break

import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

### Try batch normalization.

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
class AltConvNet(nn.Module):
    """Encoder for feature embedding"""
    def __init__(self):
        super(AltConvNet, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 64, kernel_size=(5,3), padding=(5, 3), stride=(3,1) ,dilation=(2,1)),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d(2,1))
        nn.init.xavier_uniform_(self.layer1[0].weight)
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,128,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,1)),)
        nn.init.xavier_uniform_(self.layer2[0].weight)
        self.layer3 = nn.Sequential(
                        nn.Conv2d(128,256,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,1)),)
        nn.init.xavier_uniform_(self.layer3[0].weight)
        self.fc = nn.Linear(18176, 2)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        result = self.fc(out)
        return result

def Altconv3():
    return AltConvNet()

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import  os 

trainset = ImageDataset("/content/drive/MyDrive/MATH5470/dataset/train_x.npy","/content/drive/MyDrive/MATH5470/dataset/train_y.npy")
valset = ImageDataset("/content/drive/MyDrive/MATH5470/dataset/val_x.npy","/content/drive/MyDrive/MATH5470/dataset/val_y.npy")
batch_size = 32
epoch = 15
lr = 1e-3
wd =  5e-4
model_name = 'weight.pth'
train_loader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True, num_workers=4,
                pin_memory=True, drop_last=True)
val_loader = DataLoader(dataset=valset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
device = 'cuda'
model = Altconv3().cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break

import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

### Try different activation function.

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
class AltConvNet(nn.Module):
    """Encoder for feature embedding"""
    def __init__(self):
        super(AltConvNet, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 64, kernel_size=(5,3), padding=(5, 3), stride=(3,1) ,dilation=(2,1)),
                        nn.BatchNorm2d(64),
                        nn.ReLU(),
                        nn.MaxPool2d(2,1))
        nn.init.xavier_uniform_(self.layer1[0].weight)
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,128,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(128),
                        nn.ReLU(),
                        nn.MaxPool2d((2,1)),)
        nn.init.xavier_uniform_(self.layer2[0].weight)
        self.layer3 = nn.Sequential(
                        nn.Conv2d(128,256,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(256),
                        nn.ReLU(),
                        nn.MaxPool2d((2,1)),)
        nn.init.xavier_uniform_(self.layer3[0].weight)
        self.fc = nn.Linear(18176, 2)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        result = self.fc(out)
        return result

def Altconv3():
    return AltConvNet()

In [ ]:
device = 'cuda'
model = Altconv3().cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break

import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

### Try different max-pool size.

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
class AltConvNet(nn.Module):
    """Encoder for feature embedding"""
    def __init__(self):
        super(AltConvNet, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 64, kernel_size=(5,3), padding=(5, 3), stride=(3,1) ,dilation=(2,1)),
                        nn.BatchNorm2d(64),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d(2,2))
        nn.init.xavier_uniform_(self.layer1[0].weight)
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,128,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(128),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,2)),)
        nn.init.xavier_uniform_(self.layer2[0].weight)
        self.layer3 = nn.Sequential(
                        nn.Conv2d(128,256,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(256),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,2)),)
        nn.init.xavier_uniform_(self.layer3[0].weight)
        self.fc = nn.Linear(2816, 2)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        result = self.fc(out)
        return result

def Altconv3():
    return AltConvNet()

In [ ]:
device = 'cuda'
model = Altconv3().cuda()
optimizer=torch.optim.AdamW(model.parameters(), lr, weight_decay=wd)
evaluate_batch(model, val_loader, device)
best_acc = 0
count = 0

for i in range(1, epoch+1):
    print('Epoch : ', i)
    pretrain(model, train_loader, optimizer, device)
    val_acc = evaluate_batch(model, val_loader, device)
    print('Val Acc : ', val_acc)
    if  best_acc < val_acc:
        count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_name)
    else:
        count += 1
    if count >= 2:
        break

import warnings
warnings.filterwarnings('ignore')

file_list = os.listdir("/content/monthly_20d")
img_list = [i for i in file_list if 'dat' in i]
lab_list = [i for i in file_list if 'feather' in i]
img_list.sort()
lab_list.sort()
img_train_val, img_test = img_list[:7],img_list[7:] 
lab_train_val, lab_test = lab_list[:7],lab_list[7:] 

model.eval()
N1, N2 =0, 0
for i, j in zip(image_test, label_test):
    image_test_arr=np.memmap("/content/monthly_20d/"+i, dtype=np.uint8, mode='r').reshape(
                                (-1, 64, 60))
    label_test_arr=pd.read_feather("/content/monthly_20d/"+j)['Ret_20d'].values
    testset = ImgDataset(image_test_arr, label_test_arr)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)
    test_acc = evaluate_batch(model, test_loader, device)
    N2 += len(testset)
    N1 += (test_acc *len(testset))
print('Test Acc : ', test_acc)

## Grad-CAM and data visulization

In [ ]:
from torch.utils.data import Dataset, DataLoader
from utils import *
import pandas as pd
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.functional as F
import numpy as np
import random



### Grad-CAM 
please first install the supporting pacakge:

pip install grad-cam

In [ ]:
#pip install grad-cam
from pytorch_grad_cam import GradCAM # or ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad

from pytorch_grad_cam.utils.image import show_cam_on_image


batch_size = 64
epoch = 15
lr = 1e-3
wd =  5e-4

train_x  = np.load('dataset/train_x.npy')
train_y = np.load('dataset/train_y.npy')
val_x , val_y = np.load('dataset/val_x.npy'), np.load('dataset/val_y.npy')
trainset = dataset.NpyDataset(train_x , train_y )
valset = dataset.NpyDataset(val_x , val_y)
model_name = 'weight.pth'
train_loader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True, num_workers=4,
                pin_memory=True, drop_last=True)
val_loader = DataLoader(dataset=valset, batch_size=batch_size, shuffle=False, num_workers=4,
                pin_memory=True, drop_last=True)

device = 'cpu'

#load the trained model. Here we assume the weight has been saved
model_name = "weight.pth"
model = conv3()
model.load_state_dict(torch.load(model_name,map_location=torch.device('cpu')))
model.eval()


In [ ]:
# find 5 images that are classified as 1 or 0



classification_object = 0 # or let classification_object = 1 to get another bunch of images
image_list = []
label_list = []
counter = 0
# better shuffle the data loader
for iter, pack in enumerate(val_loader):
    data, target = pack[0].to(device), pack[1].to(device)
    #targets = target.long()
    logits = model(data)
    _, pred = logits.max(1)
    class_list = data[pred==classification_object,]
    if class_list.size(0) > 0:
        obj = class_list[random.randrange(class_list.size(0)),]
        counter = counter+1
    image_list.append(obj)
    if counter >= 5:
        break

image_list = torch.stack(image_list)


input_tensors = image_list
    

# Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

# Construct the CAM object once, and then re-use it on many images:
aug_smooth = True
eigen_smooth = True


target_layer_1 = [model.layer1[-1]]
cam_1 = GradCAM(model=model, target_layers=target_layer_1, use_cuda=False)
# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam_1 = cam_1(input_tensor=input_tensors, targets=None ,aug_smooth=aug_smooth,eigen_smooth=eigen_smooth)

#visualization = show_cam_on_image(input_tensors, grayscale_cam, use_rgb=True)

target_layer_2 = [model.layer2[-1]]
cam_2 = GradCAM(model=model, target_layers=target_layer_2, use_cuda=False)
# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam_2 = cam_2(input_tensor=input_tensors, targets=None ,aug_smooth=aug_smooth,eigen_smooth=eigen_smooth)


target_layer_3 = [model.layer3[-1]]
cam_3 = GradCAM(model=model, target_layers=target_layer_3, use_cuda=False)
# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam_3 = cam_3(input_tensor=input_tensors, targets=None ,aug_smooth=aug_smooth,eigen_smooth=eigen_smooth)

camera = torch.cat((image_list[:,0,], torch.from_numpy(grayscale_cam_1),torch.from_numpy(grayscale_cam_2),torch.from_numpy(grayscale_cam_3)),0)  


### Show images

In [ ]:
# show multiple images


fig = plt.figure(figsize=(10, 10))
columns = 5
rows = 4
for i in range(0, columns*rows ):
    fig.add_subplot(rows, columns, i+1)
    plt.axis('off')
    if i<= 4:
        plt.imshow(camera[i,],cmap="gray")
    else:
        plt.imshow(camera[i,],cmap="Blues_r")
plt.savefig('4x5.png',format='png')
plt.show()

## CNN regression model

In [ ]:
# define constants

MODEL_NAME = 'model.pth'
BATCH_SIZE = 128
WEIGHT_DECAY = 5E-4
EPOCH = 15
TRAIN_DEVICE = 'cuda'
IMAGE_NAME = 'both.png'
SAVE_NAME = 'grad_cam.png'
VISUALIZATION_DEVICE = 'cpu'

RAW_DATA_PATH = 'monthly_20d'
#COOK_DATA_PATH = 'cookedData'
COOK_DATA_PATH = 'gdrive/MyDrive/Math5470Project'
VISUALIZATION_PATH = 'pic'
BINARY_CLASSIFICATION = False
DEBUG = False
LR_INITIAL = 1E-3

In [ ]:
# load the data

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch

class TrendDataset(Dataset):
    def __init__(self, data_file, label_file):
        self.data = data_file
        self.label = label_file

    def __getitem__(self, index):
        data = self.data[index]
        label = self.label[index]
        
        if BINARY_CLASSIFICATION:
            label = np.where(label > 0, 1, 0)
            label = torch.from_numpy(label).float()
        else :
            data = np.where(data == 255, 1, data)

        data = torch.from_numpy(data.copy()).float()
        data = data.unsqueeze(0)

        return data, label
    
    def __len__(self):
        return len(self.data)


In [ ]:
## regression CNN model

import torch
from torch import nn
import torch.nn.functional as F
class ConvNet(nn.Module):
    """Encoder for feature embedding"""
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 64, kernel_size=(5,3), padding=(5, 3), stride=(3,1) ,dilation=(2,1)),
                        nn.BatchNorm2d(64),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d(2,1))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,128,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(128),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,1)),)
        self.layer3 = nn.Sequential(
                        nn.Conv2d(128,256,kernel_size=(5,3),padding=(5,3), stride=(3,1), dilation=(2,1)),
                        nn.BatchNorm2d(256),
                        nn.LeakyReLU(0.1),
                        nn.MaxPool2d((2,1)),)
        self.dropout = nn.Dropout(p=0.5)

        if BINARY_CLASSIFICATION :
            self.fc = nn.Linear(18176, 2)
        else :
            self.fc = nn.Linear(18176, 1)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        result = self.fc(out)
        return result

### Pretraining

In [ ]:
## new pretrain function for regression
import numpy as np
import random
import os
from copy import deepcopy



#class Averager():
#
#    def __init__(self):
#        self.n = 0
#        self.v = 0
#
#    def add(self, x):
#        self.v = (self.v * self.n + x) / (self.n + 1)
#        self.n += 1
#
#    def item(self):
#        return self.v

def pretrain(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    #loss_avg = Averager()
    for batch_idx, batch in enumerate(loader):
        model.zero_grad()
        optimizer.zero_grad()
        train_inputs, train_targets = batch[0], batch[1]
        if BINARY_CLASSIFICATION:
          train_targets = train_targets.long()
        else:
          train_targets = train_targets.float()
        train_inputs = train_inputs.to(device=device)
        train_targets = train_targets.to(device=device)
        train_logits = model(train_inputs)

        if BINARY_CLASSIFICATION:
          loss = nn.CrossEntropyLoss()(train_logits, train_targets)
        else :
          train_targets = torch.nan_to_num(train_targets)
          loss = nn.MSELoss()(train_logits.reshape(128,), train_targets)
        total_loss += loss.item()
        #loss_avg.add(loss.item())
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(loader)
    print("Train Loss %.4f" % avg_loss)
    return avg_loss

def evaluate_batch(model, data_loader, device):
    model.eval()
    correct = num = count = 0
    total_loss = total_r_square = 0
    for iter, pack in enumerate(data_loader):
        data, target = pack[0].to(device), pack[1].to(device)
        logits = model(data)
        num += data.shape[0]
        count += 1
        if BINARY_CLASSIFICATION:
            _, pred = logits.max(1)
            correct += pred.eq(target).sum().item()
        else:
            target = torch.nan_to_num(target)
            loss = nn.MSELoss()(logits.reshape(128,), target)
            total_loss += loss.item()
            r_square = 1 - torch.sum(torch.square(torch.sub(target,logits))) / torch.sum(torch.square(target - target.mean()))
            total_r_square += r_square.item()

    torch.cuda.empty_cache()
    model.train()
    
    if BINARY_CLASSIFICATION:
        return correct / num
    else:
        print('avg r square:', total_r_square / count)
        return total_loss / count

### Training

In [ ]:
from torch.utils.data import DataLoader
import pandas as pd
#from model import ConvNet


def trainModel():
    train_x, train_y = np.load(COOK_DATA_PATH+'/train_x.npy'), np.load(COOK_DATA_PATH+'/train_y.npy')
    validate_x, validate_y = np.load(COOK_DATA_PATH+'/val_x.npy'), np.load(COOK_DATA_PATH+'/val_y.npy')
    train_dataset = TrendDataset(train_x, train_y)
    validate_dataset = TrendDataset(validate_x, validate_y)
    train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4,
                              pin_memory=True, drop_last=True)
    val_loader = DataLoader(dataset=validate_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4,
                            pin_memory=True, drop_last=True)

    model = ConvNet().cuda()
    optimizer=torch.optim.AdamW(model.parameters(), LR_INITIAL, weight_decay=WEIGHT_DECAY)
    evaluate_batch(model, val_loader, TRAIN_DEVICE)
    best_acc = 0
    count = 0

    file_list = os.listdir(os.path.join(os.getcwd(), RAW_DATA_PATH))
    img_list = [i for i in file_list if 'dat' in i]
    lab_list = [i for i in file_list if 'feather' in i]
    img_list.sort()
    lab_list.sort()

    IMAGE_WIDTH = {5: 15, 20: 60, 60: 180}
    IMAGE_HEIGHT = {5: 32, 20: 64, 60: 96}

    printed = False
    for i in range(1, EPOCH + 1):
        print('Epoch : ', i)
        pretrain(model, train_loader, optimizer, TRAIN_DEVICE)
        if printed == False:
            print(val_loader)
            printed = True
        validate_acc = evaluate_batch(model, val_loader, TRAIN_DEVICE)
        print('Validation Acc : ', validate_acc)
        print('best_acc: ', best_acc)
        if BINARY_CLASSIFICATION:
            if  best_acc < validate_acc:
                count = 0
                best_acc = validate_acc
                torch.save(model.state_dict(), MODEL_NAME)
            else:
                count += 1
        else:
            if  best_acc > validate_acc or best_acc == 0:
                count = 0
                best_acc = validate_acc
                torch.save(model.state_dict(), MODEL_NAME)
            else:
                count += 1
                
        if count >= 2:
            break

    printed = False
    model.load_state_dict(torch.load(MODEL_NAME))
    model.eval()
    for i, j in zip(img_test, lab_test):
        img_test_arr = np.memmap(os.path.join(os.getcwd(), RAW_DATA_PATH,i), dtype=np.uint8, mode='r').reshape(
                                    (-1, IMAGE_HEIGHT[20], IMAGE_WIDTH[20]))
        lab_test_arr = pd.read_feather(os.path.join(os.getcwd(), RAW_DATA_PATH,j))['Ret_20d'].values
        testset = TrendDataset(img_test_arr, lab_test_arr)
        test_loader = DataLoader(dataset=testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4,
                                 pin_memory=True, drop_last=True)
        if printed == False:
            print(test_loader)
            printed = True
        test_acc = evaluate_batch(model, test_loader, TRAIN_DEVICE)
    print('Test Acc : ', test_acc)


In [ ]:
trainModel() # 3 layer, LE activation, no BN, dropout = 0.5, or try different constant